### Import Libraries

In [1]:
import great_expectations as gx
import pandas as pd

### Creating GX Context 

In [2]:
context = gx.get_context(mode="ephemeral")


### Creating DataSource

In [3]:
data_source_name = 'temperature_data'
data_source = context.data_sources.add_pandas(name=data_source_name)

### Creating DataAsset


In [4]:
data_asset_name = 'temperature_entity_asset'
data_asset = data_source.add_dataframe_asset(name=data_asset_name)

### Creating BatchDefinition

In [5]:
batch_definition_name = 'full_batch'
batch_definition = data_asset.add_batch_definition_whole_dataframe(batch_definition_name)

### Creating Expectations

In [6]:
expectation_temp = gx.expectations.ExpectColumnValuesToBeBetween(column="Temperature",
                                                                 max_value=45, min_value=40)

In [7]:
expectation_city = gx.expectations.ExpectColumnDistinctValuesToBeInSet(column="City",
                                                                       value_set=["Mumbai", "Delhi"])

### Creating ExpectationSuite and adding Expectations

In [8]:
expectation_suite_name = "temperature_data_suite"
expectation_suite_ref = gx.ExpectationSuite(name=expectation_suite_name)
expectation_suite = context.suites.add(expectation_suite_ref)

In [9]:
#Adding Expectation to ExpectationSuite    
expectation_suite.add_expectation(expectation_temp)
expectation_suite.add_expectation(expectation_city)

ExpectColumnDistinctValuesToBeInSet(id='4f652e1c-9115-4f3b-b13a-80b12e7a59a1', meta=None, notes=None, result_format=<ResultFormat.BASIC: 'BASIC'>, description=None, catch_exceptions=False, rendered_content=None, windows=None, batch_id=None, column='City', row_condition=None, condition_parser=None, value_set=['Mumbai', 'Delhi'])

### Creating Validation Definition

In [10]:
validation_def_name = "temperature_data_validation"

validation_definition_ref = gx.ValidationDefinition(data=batch_definition,
                                                    suite=expectation_suite,
                                                    name=validation_def_name)

In [11]:
validation_definition = context.validation_definitions.add(validation_definition_ref)

### Creating Checkpoint

In [12]:
checkpoint_name = "temperature_run_checkpoint"
checkpoint_to_add = gx.Checkpoint(
    name=checkpoint_name,
    validation_definitions=[validation_definition],
    actions=[],
    result_format="COMPLETE",  # options : BOOLEAN_ONLY, BASIC, SUMMARY, COMPLETE
)


In [13]:
checkpoint = context.checkpoints.add(checkpoint_to_add)


### Reading Data in Pandas DataFrame

In [14]:
data_df = pd.read_csv('temperature.csv')
data_df.head()


,Date,City,Temperature
0,8/1/2024,Mumbai,28
1,8/2/2024,Mumbai,30
2,8/3/2024,Mumbai,32
3,8/4/2024,Mumbai,31
4,8/5/2024,Mumbai,33


### Creating BatchParameter and running Validation

In [15]:
batch_parameters = {"dataframe": data_df}


In [16]:
validation_results = checkpoint.run(batch_parameters=batch_parameters)


Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

### Displaying Validation Result

In [17]:
print(validation_results)


run_id={"run_name": null, "run_time": "2025-06-18T12:21:10.061862+06:00"} run_results={ValidationResultIdentifier::temperature_data_suite/__none__/20250618T062110.061862Z/temperature_data-temperature_entity_asset: {
  "success": false,
  "results": [
    {
      "success": false,
      "expectation_config": {
        "type": "expect_column_values_to_be_between",
        "kwargs": {
          "batch_id": "temperature_data-temperature_entity_asset",
          "column": "Temperature",
          "min_value": 40.0,
          "max_value": 45.0
        },
        "meta": {},
        "id": "04a20f0b-35a4-461b-827d-2ce91d19dc10"
      },
      "result": {
        "element_count": 62,
        "unexpected_count": 62,
        "unexpected_percent": 100.0,
        "partial_unexpected_list": [
          28,
          30,
          32,
          31,
          33,
          30,
          29,
          31,
          33,
          32,
          29,
          31,
          30,
          32,
          34,
